# Prepare Data for Analysis

In [21]:
# Import necessary libraries
import numpy as np
import pandas as pd
from scipy.optimize import minimize

In [22]:
# Load the data from a CSV file into a DataFrame
df = pd.read_csv('../Resources/all_data.csv')

C:\Users\thoma\AppData\Local\Temp\ipykernel_22544\3448858262.py:1: DtypeWarning: Columns (60) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../Resources/all_data.csv')


In [23]:
# Sort values by player id and season
df = df.sort_values(by=['IDfg', 'Season'], ascending=[True, True])

In [24]:
df.head()

,IDfg,Season,Name,Team,Age,G,AB,PA,H,1B,...,maxEV,HardHit,HardHit%,Events,CStr%,CSW%,xBA,xSLG,xwOBA,L-WAR
67030,1,2002,Alfredo Amezaga,ANA,24,12,13,13,7,5,...,NaN,NaN,NaN,0,0.180,0.260,NaN,NaN,NaN,0.2
68116,1,2003,Alfredo Amezaga,ANA,25,37,105,120,22,15,...,NaN,NaN,NaN,0,0.210,0.303,NaN,NaN,NaN,-0.2
68714,1,2004,Alfredo Amezaga,ANA,26,73,93,105,15,11,...,NaN,NaN,NaN,0,0.175,0.282,NaN,NaN,NaN,-0.3
70296,1,2005,Alfredo Amezaga,- - -,27,6,8,9,2,2,...,NaN,NaN,NaN,0,0.138,0.172,NaN,NaN,NaN,0.0
70920,1,2006,Alfredo Amezaga,FLA,28,132,334,378,87,72,...,NaN,NaN,NaN,0,0.188,0.256,NaN,NaN,NaN,1.1


In [25]:
# Drop columns with any missing values
df = df.dropna(axis=1)

In [26]:
# Create 2025 rows of empty data to add projections to

# Select 2024 players
players_2024 = df[df['Season'] == 2024]['Name'].unique()
ID_2024 = df[df['Season'] == 2024]['IDfg'].unique()

# Step 2: Create a new DataFrame with a 2025 row for each player from the 2024 list
new_rows = pd.DataFrame({
    'IDfg': ID_2024,
    'Name': players_2024,
    'Season': 2025
})

# Add blank values for columns 
for col in df.columns:
    if col not in new_rows.columns:
        new_rows[col] = None

# Step 3: Concatenate the new rows to the original DataFrame
df = pd.concat([df, new_rows], ignore_index=True)

df.tail()

C:\Users\thoma\AppData\Local\Temp\ipykernel_22544\3541928745.py:17: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, new_rows], ignore_index=True)


,IDfg,Season,Name,Team,Age,G,AB,PA,H,1B,...,wRC+,BsR,Def,wSB,Age Rng,Off,Lg,BB%+,Events,L-WAR
88103,33189,2025,Nolan Schanuel,None,None,None,None,None,None,None,...,None,NaN,NaN,NaN,None,NaN,NaN,None,None,NaN
88104,33266,2025,Jacob Wilson,None,None,None,None,None,None,None,...,None,NaN,NaN,NaN,None,NaN,NaN,None,None,NaN
88105,33333,2025,Wyatt Langford,None,None,None,None,None,None,None,...,None,NaN,NaN,NaN,None,NaN,NaN,None,None,NaN
88106,33541,2025,Dylan Crews,None,None,None,None,None,None,None,...,None,NaN,NaN,NaN,None,NaN,NaN,None,None,NaN
88107,33824,2025,Jung Hoo Lee,None,None,None,None,None,None,None,...,None,NaN,NaN,NaN,None,NaN,NaN,None,None,NaN


In [27]:
# Drop specific columns by name
columns_to_drop = ['Age Rng', 'Events']  # Specify the columns to drop
df = df.drop(columns=columns_to_drop)

In [28]:
# Get the list of columns
columns_list = df.columns.tolist()

print(columns_list)

['IDfg', 'Season', 'Name', 'Team', 'Age', 'G', 'AB', 'PA', 'H', '1B', '2B', '3B', 'HR', 'R', 'RBI', 'BB', 'SH', 'SB', 'AVG', 'BB%', 'OBP', 'SLG', 'OPS', 'ISO', 'wOBA', 'wRAA', 'wRC', 'Bat', 'Rep', 'Pos', 'RAR', 'WAR', 'Spd', 'wRC+', 'BsR', 'Def', 'wSB', 'Off', 'Lg', 'BB%+', 'L-WAR']


In [29]:
# Columns to exclude
exclude = ['IDfg', 'Season', 'Name', 'Team', 'Age']

# Create a new list excluding certain values
filtered_columns = [col for col in columns_list if col not in exclude]

print(filtered_columns)

['G', 'AB', 'PA', 'H', '1B', '2B', '3B', 'HR', 'R', 'RBI', 'BB', 'SH', 'SB', 'AVG', 'BB%', 'OBP', 'SLG', 'OPS', 'ISO', 'wOBA', 'wRAA', 'wRC', 'Bat', 'Rep', 'Pos', 'RAR', 'WAR', 'Spd', 'wRC+', 'BsR', 'Def', 'wSB', 'Off', 'Lg', 'BB%+', 'L-WAR']


In [30]:
# Create columns to put previous years data into
for value in filtered_columns:
    df[f'1Prev_{value}'] = df.groupby('IDfg')[value].shift(1)
    df[f'2Prev_{value}'] = df.groupby('IDfg')[value].shift(2)
    df[f'3Prev_{value}'] = df.groupby('IDfg')[value].shift(3)

C:\Users\thoma\AppData\Local\Temp\ipykernel_22544\2058564811.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'1Prev_{value}'] = df.groupby('IDfg')[value].shift(1)
C:\Users\thoma\AppData\Local\Temp\ipykernel_22544\2058564811.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'2Prev_{value}'] = df.groupby('IDfg')[value].shift(2)
C:\Users\thoma\AppData\Local\Temp\ipykernel_22544\2058564811.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which h

In [31]:
# Add previous three years data to columns
for value in filtered_columns:
    league_avg = df.groupby('Season')[value].mean().reset_index()
    league_avg = league_avg.rename(columns={value: f'League_Avg_{value}'})
    league_avg[f'Prev_League_Avg_{value}'] = league_avg[f'League_Avg_{value}'].shift(1)
    df = df.merge(league_avg[['Season', f'Prev_League_Avg_{value}']], on='Season', how='left')
    

In [32]:
df.head()

,IDfg,Season,Name,Team,Age,G,AB,PA,H,1B,...,Prev_League_Avg_WAR,Prev_League_Avg_Spd,Prev_League_Avg_wRC+,Prev_League_Avg_BsR,Prev_League_Avg_Def,Prev_League_Avg_wSB,Prev_League_Avg_Off,Prev_League_Avg_Lg,Prev_League_Avg_BB%+,Prev_League_Avg_L-WAR
0,1,2002,Alfredo Amezaga,ANA,24,12,13,13,7,5,...,0.588374,2.617284,45.798354,0.003292,0.478909,0.003292,-0.964300,0.475000,70.820988,0.588374
1,1,2003,Alfredo Amezaga,ANA,25,37,105,120,22,15,...,0.601475,2.539937,45.566913,-0.000421,0.374605,0.001475,-0.907165,0.527713,70.727081,0.601475
2,1,2004,Alfredo Amezaga,ANA,26,73,93,105,15,11,...,0.590796,2.518304,47.617373,0.002689,0.397622,0.002896,-0.928645,0.521406,72.698035,0.590796
3,1,2005,Alfredo Amezaga,- - -,27,6,8,9,2,2,...,0.591589,2.474143,39.111111,0.002181,0.398235,0.001869,-0.965005,0.560021,70.193146,0.591589
4,1,2006,Alfredo Amezaga,FLA,28,132,334,378,87,72,...,0.608289,2.477790,47.773645,0.003932,0.369182,0.003826,-0.888629,0.513921,72.802338,0.608289
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88103,33189,2025,Nolan Schanuel,None,None,None,None,None,None,None,...,0.875422,3.857143,80.602151,0.001690,-0.894316,0.001690,0.030261,0.861751,93.483871,0.875115
88104,33266,2025,Jacob Wilson,None,None,None,None,None,None,None,...,0.875422,3.857143,80.602151,0.001690,-0.894316,0.001690,0.030261,0.861751,93.483871,0.875115
88105,33333,2025,Wyatt Langford,None,None,None,None,None,None,None,...,0.875422,3.857143,80.602151,0.001690,-0.894316,0.001690,0.030261,0.861751,93.483871,0.875115
88106,33541,2025,Dylan Crews,None,None,None,None,None,None,None,...,0.875422,3.857143,80.602151,0.001690,-0.894316,0.001690,0.030261,0.861751,93.483871,0.875115


In [33]:
df.to_csv('../Resources/properly_formatted_data.csv', index=False)